# 2022-09-30 • Use parametric STA model for connection testing

## Imports

In [1]:
#

In [2]:
using MyToolbox

In [4]:
using VoltoMapSim

## Params

In [5]:
p = get_params(
    duration = 10minutes,
    p_conn = 0.04,
    g_EE = 1,
    g_EI = 1,
    g_IE = 4,
    g_II = 4,
    ext_current = Normal(-0.5 * pA/√seconds, 5 * pA/√seconds),
    E_inh = -80 * mV,
    record_v = [1:40; 801:810],
);

## Load STA's

(They're precalculated).

In [6]:
out = cached_STAs(p);

Loading cached output from `C:\Users\tfiers\.phdcache\calc_all_STAs\b9353bdd11d8b8cb.jld2` … done (12.9 s)


In [7]:
(conns, STAs, shuffled_STAs) = out;

## Test one conn.

In [8]:
conn = (conns.pre .=> conns.post)[1]

139 => 1

In [10]:
fit = fit_STA(STAs[conn], p)

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([0.0131, 0.0131, 0.0129, 0.08, 0.08, 0, 0.000357], [4.5E-05, 3.61E-05, 3.74E-05, 5.71E-05, 4.09E-05, 2.78E-05, 2.26E-05, -8.39E-06, -4.39E-05, -1.67E-05  …  -6.25E-05, -8E-05, -7.93E-05, -6.69E-05, -8.25E-05, -0.000109, -0.000101, -0.000101, -0.000119, -0.000119], [0.0105 -0.00452 … 9.85E-05 -0.35; 0.0105 -0.00452 … 9.82E-05 -0.35; … ; 0.011 -0.00273 … -3.13E-05 -0.327; 0.011 -0.00274 … -3.12E-05 -0.327], true, Float64[])